# Google Connection

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# Installs

In [2]:
!pip install sentence-transformers
!pip install rdflib

import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

! pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 4.9 MB 38.0 MB/s 
     |████████████████████████████████| 1.3 MB 34.3 MB/s 
     |████████████████████████████████| 163 kB 43.9 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=da6f42d76642640da843bcab38af25ff2251c04374bf075d73f605f99e70e8b1
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 5.1 MB/s 
     |████████████████████████████████| 41 kB 536 kB/s 
1.12.1+cu113
     |████████████████████████████████| 7.9 MB 5.4 MB/s 
  

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch.nn.functional as F
import os

import logging
import os.path as osp
import ssl
import urllib.request
from rdflib import Graph, Literal, URIRef
import random
import numpy as np
from collections import Counter

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv

import xml.etree.ElementTree as etree  # from lxml import etree #for parser
from io import BytesIO
import logging
import tempfile
from six.moves import urllib
from xml.sax.saxutils import quoteattr, escape

from torchmetrics.functional import pairwise_cosine_similarity

In [4]:
import sys
main_files_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/MAGNET"

sys.path.append(os.path.abspath(main_files_path))

from load_data import download_url, create_pyg_data, create_fake_data
from stuff_for_model import calc_acc
from interpretation import count_occurences, check_real_val_of_both, check_single_left_vals, check_single_right_vals, save_output

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Code I don't touch

In [6]:
def parse_mapping_from_file(source, parse_cell_extensions=False):
    """
    Parses a alignment from a filename or file object.
    :param source: is a filename or file object containing a alignment in alignment format
    :param parse_cell_extensions: if true, also parses the cell extensions
    :return: (alignment: list of (source, target, relation, confidence, extensions - which are parsed only when parse_cell_extensions is True),
    onto1 as (id, url, formalismName, formalismURI),
    onto2 similar to onto1,
    extension (iterable of key, values)
    )
    """
    handler = AlignmentHandler()
    etree.parse(source, etree.XMLParser(target=handler))
    if parse_cell_extensions == False:
        remove_cell_extensions(handler.alignment)
    return handler.alignment, handler.onto1, handler.onto2, handler.extension

class AlignmentHandler(object):
    def __init__(self):
        self.base = "{http://knowledgeweb.semanticweb.org/heterogeneity/alignment}"
        self.rdf = "{http://www.w3.org/1999/02/22-rdf-syntax-ns#}"
        self.text = ""
        self.alignment = []
        self.one_cell = ["", "", "", "", {}]
        self.extension = {}
        self.onto1 = ""
        self.onto2 = ""
        self.onto_temp = ["", ""]
        self.in_cell = False
        self.used_tags = set(
            [
                self.base + name
                for name in [
                    "entity1",
                    "entity2",
                    "relation",
                    "measure",
                    "Cell",
                    "map",
                    "Alignment",
                    "xml",
                    "level",
                    "type",
                    "onto1",
                    "onto2",
                    "Ontology",
                    "location",
                    "formalism",
                    "Formalism",
                ]
            ]
        )
        self.used_tags.add(self.rdf + "RDF")

    def start(self, name, attrs):
        if name == self.base + "entity1":
            self.one_cell[0] = attrs[self.rdf + "resource"]  # .encode('utf-8')
        elif name == self.base + "entity2":
            self.one_cell[1] = attrs[self.rdf + "resource"]  # .encode('utf-8')
        elif name == self.base + "Ontology":
            self.onto_temp[0] = attrs[self.rdf + "about"]  # .encode('utf-8')
        elif name == self.base + "Cell":
            self.in_cell = True
        self.text = ""

    def end(self, name):
        if name == self.base + "relation":
            self.one_cell[2] = self.text.strip()
        elif name == self.base + "measure":
            self.one_cell[3] = self.text.strip()
        elif name == self.base + "Cell":
            self.alignment.append(self.one_cell)
            self.one_cell = ["", "", "", "", {}]
            self.in_cell = False
        elif name == self.base + "location":
            self.onto_temp[1] = self.text.strip()
        elif name == self.base + "onto1":
            if self.onto_temp[0] == "" and self.onto_temp[1] == "":
                self.onto_temp[0] = self.text.strip()
            self.onto1 = list(self.onto_temp)
        elif name == self.base + "onto2":
            if self.onto_temp[0] == "" and self.onto_temp[1] == "":
                self.onto_temp[0] = self.text.strip()
            self.onto2 = list(self.onto_temp)
        elif name not in self.used_tags:
            key = name.replace("{","",1).replace("}","",1) #name[name.index("}") + 1 :]
            if self.in_cell:
                self.one_cell[4][key] = self.text
            else:
                self.extension[key] = self.text

    def data(self, chars):
        self.text += chars

    def close(self):
        pass

def remove_cell_extensions(alignment):
    for c in alignment:
        c.pop()

In [7]:
def _get_alignment(alignment, resource_position_map_one, resource_position_map_two):
    left_indices_list = []
    right_indices_list = []
    for src, tgt, rel, conf in alignment:
        left_indices_list.append(resource_position_map_one[src])
        right_indices_list.append(resource_position_map_two[tgt])
    left_indices = torch.tensor(left_indices_list)
    right_indices = torch.tensor(right_indices_list)
    return left_indices, left_indices_list, right_indices, right_indices_list


def _train_test_val_split(left_indices_list, right_indices_list):
    train_indices_left = left_indices_list[: int(len(left_indices_list) * 0.7)]
    train_indices_right = right_indices_list[: int(len(right_indices_list) * 0.7)]
    test_indices_left = left_indices_list[int(len(left_indices_list) * 0.7):int(len(left_indices_list) * 0.9)]
    test_indices_right = right_indices_list[int(len(right_indices_list) * 0.7):int(len(right_indices_list) * 0.9)]
    val_indices_left = left_indices_list[int(len(left_indices_list) * 0.9):]
    val_indices_right = right_indices_list[int(len(right_indices_list) * 0.9):]
    train_set_left = torch.tensor(train_indices_left)
    test_set_left = torch.tensor(test_indices_left)
    val_set_left = torch.tensor(val_indices_left)
    train_set_right = torch.tensor(train_indices_right)
    test_set_right = torch.tensor(test_indices_right)
    val_set_right = torch.tensor(val_indices_right)
    return test_set_left, test_set_right, train_set_left, train_set_right, val_set_left, val_set_right


# Create Data

In [8]:
# Main
def create_pyg_data(kg_file_one, kg_file_two, alignment_file):

    pred_dict, graph_one, graph_two = get_pred_dict(kg_file_one, kg_file_two)
    x_prepared_embs_one, x_embs_and_literals_one, res_edge_index_one, res_edge_type_one, all_edge_index_one, all_edge_type_one, edge_attr_embs_res_tensor_one, edge_attr_embs_all_tensor_one, resource_position_map_one = parse_kg_file(graph_one, pred_dict)
    x_prepared_embs_two, x_embs_and_literals_two, res_edge_index_two, res_edge_type_two, all_edge_index_two, all_edge_type_two, edge_attr_embs_res_tensor_two, edge_attr_embs_all_tensor_two, resource_position_map_two  = parse_kg_file(graph_two, pred_dict)

    alignment, onto1, onto2, extension = parse_mapping_from_file(alignment_file)

    left_indices, left_indices_list, right_indices, right_indices_list = _get_alignment(alignment,
                                                                                        resource_position_map_one,
                                                                                        resource_position_map_two)

    test_set_left, test_set_right, train_set_left, train_set_right, val_set_left, val_set_right = _train_test_val_split(
        left_indices_list, right_indices_list)

    return Data(x_one=x_prepared_embs_one, edge_index_one=res_edge_index_one,
                edge_type_one=res_edge_type_one, edge_attr_emb_one=edge_attr_embs_res_tensor_one,
                map_one= edge_attr_embs_res_tensor_one, res_map_one=resource_position_map_one,
                x_two=x_prepared_embs_two, edge_index_two=res_edge_index_two,
                edge_type_two=res_edge_type_two, edge_attr_emb_two=edge_attr_embs_all_tensor_two,
                map_two=edge_attr_embs_res_tensor_two, res_map_two=resource_position_map_two,
                left_indices=left_indices, right_indices=right_indices, train_set_left=train_set_left, test_set_left=test_set_left, val_set_left=val_set_left,
                train_set_right=train_set_right, test_set_right=test_set_right, val_set_right=val_set_right
                ), Data(x_one=x_embs_and_literals_one, edge_index_one=all_edge_index_one,
                edge_type_one=all_edge_type_one, edge_attr_emb_one=edge_attr_embs_all_tensor_one,
                map_one= edge_attr_embs_res_tensor_one, res_map_one=resource_position_map_one,
                x_two=x_embs_and_literals_two, edge_index_two=all_edge_index_two,
                edge_type_two=all_edge_type_two, edge_attr_emb_two=edge_attr_embs_all_tensor_two,
                map_two=edge_attr_embs_res_tensor_two, res_map_two=resource_position_map_two,
                left_indices=left_indices, right_indices=right_indices, train_set_left=train_set_left, test_set_left=test_set_left, val_set_left=val_set_left,
                train_set_right=train_set_right, test_set_right=test_set_right, val_set_right=val_set_right
                )

In [9]:
def get_pred_dict(file_one, file_two):
  graph_one = create_graph(file_one)
  graph_two = create_graph(file_two)
  return create_dict(graph_one, graph_two), graph_one, graph_two


def create_graph(file):
  graph = Graph()
  return graph.parse(file)


def create_dict(graph_one, graph_two):
  pred_one = get_preds(graph_one)
  pred_two = get_preds(graph_two)
  print(f"Count of first graph ind. predicates: {len(pred_one)}")
  print(f"Count of sec. graph ind. predicates: {len(pred_two)}")
  return dict((element, index) for (index, element) in enumerate(pred_one.union(pred_two)))

def get_preds(graph):   
  set_of_diff_preds = set()
  for subj, pred, obj in graph:
    set_of_diff_preds.add(pred)
  return set_of_diff_preds

# Next part

In [10]:
def parse_kg_file(rdflib_graph, pred_dict):

    resource_position_map, attribute_position_map = _get_ressources_and_properties(rdflib_graph)
    all_edges, res_edges, all_edge_type, res_edge_type = _create_edges(rdflib_graph,
                                                                       resource_position_map,
                                                                       attribute_position_map,
                                                                       pred_dict)
    res_edge_index, res_edge_type, all_edge_index, all_edge_type = _change_datatype(res_edges, res_edge_type, all_edges, all_edge_type)

    resource_attr_map = _update_embeddings_with_literals(rdflib_graph, attribute_position_map, resource_position_map)
    x_prepared_embs = _create_embeddings_for_prepared_nodes(resource_attr_map)


    x_embs_and_literals = _create_embeddings_for_nodes_plus_literals(resource_attr_map, attribute_position_map)


    edge_attr_embs_res_tensor, edge_attr_embs_all_tensor = _create_embeddings_for_edges(pred_dict, res_edge_type, all_edge_type)

    return x_prepared_embs, x_embs_and_literals, res_edge_index, res_edge_type, all_edge_index, all_edge_type, edge_attr_embs_res_tensor, edge_attr_embs_all_tensor, resource_position_map

In [11]:
def _create_embeddings_for_edges(pred_dict, res_edge_type, all_edge_type):
  ### TODO: ACTIVATE ROW

  print('_create_embeddings_for_edges')
  properties_embedding_map = pred_dict.copy()
  for key in pred_dict.keys():
    properties_embedding_map.update({key: model.encode(key.split('/')[-1])}) # das ist bei jedem anders, ist jetzt für starwars klein
    #properties_embedding_map.update({key: model.encode(key.split('#')[-1])}) # für maus benutze ich jetzt einfach das, wird schon nicht schlechter als random sein

  edge_attr_embs_all = []
  edge_attr_embs_res = []

  positions_rev = {y: x for x, y in pred_dict.items()}

  for t in res_edge_type:
    edge_attr_embs_res.append(properties_embedding_map[positions_rev[t.item()]]) # Herausfinden, warum es hier anders ist!!!!
  for t in all_edge_type:
    edge_attr_embs_all.append(properties_embedding_map[positions_rev[t.item()]])

  edge_attr_embs_all = np.array(edge_attr_embs_all)
  edge_attr_embs_res = np.array(edge_attr_embs_res)

  edge_attr_embs_res_tensor = torch.tensor(edge_attr_embs_res, dtype=torch.float)
  edge_attr_embs_all_tensor = torch.tensor(edge_attr_embs_all, dtype=torch.float)
  return edge_attr_embs_res_tensor, edge_attr_embs_all_tensor

def _create_embeddings_for_prepared_nodes(resource_attr_map):  
  print('_create_embeddings_for_prepared_nodes')
  attributes = resource_attr_map.values()
  embeddings = model.encode([i for i in list(attributes)])
  return torch.tensor(embeddings)

def _change_datatype(res_edges, res_edge_type, all_edges, all_edge_type):  
  print('_change_datatype')
  res_edge_index_tensor = torch.tensor(res_edges, dtype=torch.long)
  res_edge_index = res_edge_index_tensor.t().contiguous()
  res_edge_type = torch.tensor(res_edge_type, dtype=torch.long)

  all_edge_index_tensor = torch.tensor(all_edges, dtype=torch.long)
  all_edge_index = all_edge_index_tensor.t().contiguous()
  all_edge_type = torch.tensor(all_edge_type, dtype=torch.long)

  return res_edge_index, res_edge_type, all_edge_index, all_edge_type

def _update_embeddings_with_literals(graph, attribute_position_map, resource_position_map):
  print('_update_embeddings_with_literals')
  attribute_position_map_rev = {y: x for x, y in attribute_position_map.items()}
  resource_attr_map = {y: x for x, y in resource_position_map.items()}

  for a, i in resource_position_map.items():
      triples_of_curr = graph.triples((URIRef(a), None, None))
      new_des = ' '
      for s, p, o in triples_of_curr:
        if type(o) == Literal:
          new_des = new_des + o + ' '
      # wir können ja sowas machen wie z.B. : nimm die letzten paar Buchstaben ohne Sonderzeichen wie z.B. "#, /" und checke ob es ein echtes Wort ist
      ### TODO: ACTIVATE ROW
      new_des = new_des + resource_attr_map[i].split('/')[-1] # Das bringt bei der Maus nichts - würde es nur schlechter machen, Abfrage überlegen!!!
      resource_attr_map.update({i: new_des})
  return resource_attr_map

def _create_edges(graph, resource_position_map, attribute_position_map, pred_dict):
  print('_create_edges')
  all_edges_list = []
  ressources_edges_list = []
  all_edge_type_list = []
  ressources_edge_type_list = []

  for subj, pred, obj in graph:
      if type(subj) is URIRef and type(obj) is URIRef:
        all_edges_list.append([resource_position_map[subj.toPython()], resource_position_map[obj.toPython()]])
        ressources_edges_list.append([resource_position_map[subj.toPython()], resource_position_map[obj.toPython()]])
        all_edge_type_list.append(pred_dict[pred])
        ressources_edge_type_list.append(pred_dict[pred])
      elif type(subj) is URIRef and type(obj) is Literal:
        all_edges_list.append([resource_position_map[subj.toPython()], attribute_position_map[str(obj.toPython()).lower().replace('_', ' ')]])
        all_edge_type_list.append(pred_dict[pred])
  return all_edges_list, ressources_edges_list, all_edge_type_list, ressources_edge_type_list


def _get_ressources_and_properties(rdflib_graph):
  print('_get_ressources_and_properties')
  resources = set()
  attributes = set()

  for subj, pred, obj in rdflib_graph:
      if type(subj) is URIRef:
          resources.add(subj.toPython())
      if type(obj) is URIRef:
          resources.add(obj.toPython())
      if type(obj) is Literal:
          attributes.add(str(obj.toPython()).lower().replace('_', ' '))

  resource_position_map = dict((element, index) for (index, element) in enumerate(resources))
  attribute_position_map = dict((element, index) for (index, element) in enumerate(attributes, len(resource_position_map))) 
  return resource_position_map, attribute_position_map

In [12]:
def _create_embeddings_for_nodes_plus_literals(resource_position_map, attribute_position_map):
  print("_create_embeddings_for_nodes_plus_literals")
  try:
    attribute_position_map['tooth']
    emb_res = torch.randn(len(resource_position_map), 384)
  except:
    res_embs = _prepare_simple_embs_for_nodes(resource_position_map)
    emb_res = _create_embs_from_simplified_nodes(res_embs)
  emb_lit = _attribute_emb(attribute_position_map)
  return torch.cat((emb_res, emb_lit), 0) 


def _prepare_simple_embs_for_nodes(resource_position_map):
  print('_prepare_simple_embs_for_nodes')
  res_embs = []
  for key in resource_position_map.keys():
    res_embs.append(resource_position_map[key].split('/')[-1])
  return res_embs


def _create_embs_from_simplified_nodes(simplified):
  print('_create_embs_from_simplified_nodes')
  embeddings_res = [model.encode(i) for i in simplified]
  return torch.tensor(np.array(embeddings_res))


def _attribute_emb(attribute_position_map):  
  print('_attribute_emb')
  attributes = attribute_position_map.keys() # Richtige Embeddings für alle Literale
  embeddings_lits = model.encode([i for i in list(attributes)])
  return torch.tensor(embeddings_lits)

# Ausprobieren von Code

In [ ]:
data_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/Data/knowledge_graphs/"

In [ ]:
#pred_dict, graph_one, graph_two = get_pred_dict(data_path + "anatomy/mouse_source.xml", data_path + "anatomy/human_target.xml")

In [ ]:
#pred_dict, graph_one, graph_two = get_pred_dict(data_path + "starwars_big/starwars_big_source.xml", data_path + "starwars_big/starwars_big_target.xml")

In [ ]:
pred_dict, graph_one, graph_two = get_pred_dict(data_path + "starwars_small/starwars_small_source.xml", data_path + "starwars_small/starwars_small_target.xml")

Count of first graph ind. predicates: 384
Count of sec. graph ind. predicates: 165


In [ ]:
x_prepared_embs, x_embs_and_literals, res_edge_index, res_edge_type, all_edge_index, all_edge_type, edge_attr_embs_res_tensor, edge_attr_embs_all_tensor, resource_position_map = parse_kg_file(graph_one, pred_dict)

_get_ressources_and_properties
_create_edges
_change_datatype
_update_embeddings_with_literals
_create_embeddings_for_prepared_nodes


KeyboardInterrupt: ignored

# Ausführen 

Wir machen das, um zu verstehen, welche Aufbereitungsmethode am Besten für die 
jeweiligen Daten funktioniert

In [ ]:
file_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/prep_data/"

In [ ]:
data_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/Data/knowledge_graphs/"

In [ ]:
preprocessed_data_path = "/content/gdrive/My Drive/1_Studium/2_Master/Master Thesis/2_Projekt/Scratch/prep_data/"

In [ ]:
#res_data, all_data = create_pyg_data(data_path + "anatomy/mouse_source.xml", data_path + "anatomy/human_target.xml", data_path + "anatomy/mouse_human_ref.xml")

Count of first graph ind. predicates: 11
Count of sec. graph ind. predicates: 11
_create_embeddings_for_edges
_create_embeddings_for_edges


In [ ]:
#torch.save(res_data, preprocessed_data_path+"/anatomy_ressource_focused.pt")

In [ ]:
#torch.save(all_data, preprocessed_data_path+"/anatomy_literal_focused.pt")

In [ ]:
#res_data, all_data = create_pyg_data(data_path + "starwars_big/starwars_big_source.xml", data_path + "starwars_big/starwars_big_target.xml", data_path + "starwars_big/starwars_big_ref.xml")

Count of first graph ind. predicates: 440
Count of sec. graph ind. predicates: 342
_get_ressources_and_properties
_create_edges
_change_datatype
_update_embeddings_with_literals
_create_embeddings_for_prepared_nodes
_create_embeddings_for_nodes_plus_literals
_attribute_emb
_create_embeddings_for_edges
_get_ressources_and_properties
_create_edges
_change_datatype
_update_embeddings_with_literals
_create_embeddings_for_prepared_nodes
_create_embeddings_for_nodes_plus_literals
_attribute_emb
_create_embeddings_for_edges


In [ ]:
#pred_dict, graph_one, graph_two = get_pred_dict(data_path + "starwars_big/starwars_big_source.xml", data_path + "starwars_big/starwars_big_target.xml")

Count of first graph ind. predicates: 440
Count of sec. graph ind. predicates: 342


In [ ]:
#torch.save(pred_dict, preprocessed_data_path+"/starwars_big_pred_dict.pt")

In [ ]:
#torch.save(graph_one, preprocessed_data_path+"/starwars_big_graph_one.pt")

In [ ]:
#torch.save(graph_two, preprocessed_data_path+"/starwars_big_graph_two.pt")

In [ ]:
graph_one = torch.load(preprocessed_data_path+"/starwars_big_graph_one.pt")

In [ ]:
graph_two = torch.load(preprocessed_data_path+"/starwars_big_graph_two.pt")

In [ ]:
pred_dict = torch.load(preprocessed_data_path+"/starwars_big_pred_dict.pt")

one

In [ ]:
#x_prepared_embs_one, x_embs_and_literals_one, res_edge_index_one, res_edge_type_one, all_edge_index_one, all_edge_type_one, edge_attr_embs_res_tensor_one, edge_attr_embs_all_tensor_one, resource_position_map_one = parse_kg_file(graph_one, pred_dict)

_get_ressources_and_properties
_create_edges
_change_datatype
_update_embeddings_with_literals
_create_embeddings_for_prepared_nodes
_create_embeddings_for_nodes_plus_literals
_attribute_emb
_create_embeddings_for_edges


In [ ]:
#starwars_big_data_res_one = Data(x_one=x_prepared_embs_one, edge_index_one=res_edge_index_one, edge_type_one=res_edge_type_one, edge_attr_embs_res_tensor_one=edge_attr_embs_res_tensor_one, resource_position_map_one=resource_position_map_one)

In [ ]:
#torch.save(starwars_big_data_res_one, preprocessed_data_path+"/starwars_big_data_res_one.pt")

In [ ]:
#starwars_big_data_all_one = Data(x_one=x_embs_and_literals_one, edge_index_one=all_edge_index_one, edge_type_one=all_edge_type_one, edge_attr_embs_res_tensor_one=edge_attr_embs_all_tensor_one, resource_position_map_one=resource_position_map_one)

In [ ]:
#torch.save(starwars_big_data_all_one, preprocessed_data_path+"/starwars_big_data_all_one.pt")

In [ ]:
#torch_res_one = torch.load(preprocessed_data_path+"/starwars_big_data_res_one.pt")

two

In [ ]:
x_prepared_embs_two, x_embs_and_literals_two, res_edge_index_two, res_edge_type_two, all_edge_index_two, all_edge_type_two, edge_attr_embs_res_tensor_two, edge_attr_embs_all_tensor_two, resource_position_map_two  = parse_kg_file(graph_two, pred_dict)

_get_ressources_and_properties
_create_edges
_change_datatype
_update_embeddings_with_literals
_create_embeddings_for_prepared_nodes
_create_embeddings_for_nodes_plus_literals
_attribute_emb


In [ ]:
starwars_big_data_res_two = Data(x_two=x_prepared_embs_two, edge_index_two=res_edge_index_two, edge_type_two=res_edge_type_two, edge_attr_embs_res_tensor_two=edge_attr_embs_res_tensor_two, resource_position_map_two=resource_position_map_two)

In [ ]:
starwars_big_data_all_two = Data(x_two=x_embs_and_literals_two, edge_index_two=all_edge_index_two, edge_type_two=all_edge_type_two, edge_attr_embs_all_tensor_two=edge_attr_embs_all_tensor_two, resource_position_map_two=resource_position_map_two)

In [ ]:
torch.save(starwars_big_data_res_two, preprocessed_data_path+"/starwars_big_data_res_two.pt")

In [ ]:
torch.save(starwars_big_data_all_two, preprocessed_data_path+"/starwars_big_data_all_two.pt")

In [ ]:
#torch.save(res_data, preprocessed_data_path+"/starwars_big_ressource_focused.pt")

In [ ]:
#torch.save(all_data, preprocessed_data_path+"/starwars_big_literal_focused.pt")